# Discovery of Frequent Itemsets and Association Rules

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

runExampleFromClass = False # True for lecture example, slide 43
s_threshold = 1000 # 2 for lecture example or 1000 for large data set 

if runExampleFromClass:
  file_source = '/content/../Example In Class.txt'
else:
  file_source = '/content/../T10I4D100K.dat'

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
lines = []
sets = []
for line in open(file_source, 'r'):
  line = line.replace("\n","").strip() 
  lines.append(line)
  sets.append(set(line.split(" ")))

In [ ]:
if runExampleFromClass:
  for s in sets:
    print(s)

1st scan

In [ ]:
#AB: Pass 1: Read baskets and count in main memory the occurrences of each individual item
itemsets = {}
#create the singletons: count how often each item occurs
for se in sets: #per each transaction set
  for itemset in se: #per each item per each set
    if itemset not in itemsets: 
        itemsets[itemset] = 1
    else: 
      itemsets[itemset] += 1

if runExampleFromClass:
  print(itemsets)

Now, apply threshold and create L1

In [ ]:
itemSetsOverThreshold = {}
singletons = []
for itemset, sup in itemsets.items():
  if sup >= s_threshold: 
    singletons.append(itemset)
    # itemSetsOverThreshold[itemset]= sup #set frozenset i.e. elements are immutable
    itemSetsOverThreshold[frozenset([itemset])]= sup #set frozenset i.e. elements are immutable

if runExampleFromClass:
  print(itemSetsOverThreshold)
  print(singletons)

Now let's do this iteratively for 3 scans

In [ ]:
import itertools as it

In [ ]:
result = []
result += [itemSetsOverThreshold] #add the singletons

resultdictionary = {}
for itemset, sup in itemSetsOverThreshold.items():
  resultdictionary[itemset] = sup 

numberOfScans = 2 #only need two more since we already did one scan earlier

In [ ]:
#Create all combination  of doubletons, tripletons and saves how frequent respective occurs over threshold
for index in range(1,numberOfScans+1): #for each scan
  combinationsets = {}

  # print(itemSetsOverThreshold)  

  #Create combination between the itemSetsOverThreshold combo[0] e.g. = {'C', 'A'} and the respecitve singletons e.g. C to  {'C','A'] or with E to {'E', 'C', 'A'}
    #save these combination with starting counter = 0
  for combo in ((itemkeys,singeltonekeys) for itemkeys in itemSetsOverThreshold.keys() for singeltonekeys in singletons):
    cand = combo[0] | frozenset([combo[1]]) #has to be frozen, otherwise unhashable in next step; union removes duplicates between sets like in the example in previous comment
    combinationsets[cand] = 0 #give 0 as starting value before counting
    #it serves as a list where we will keep track of how often candidates occur, this will be done in the next part
    
  #get all of the keys from the combination i.e. without the counter  
  comb_keys = set(combinationsets.keys())

  #Create combination of the transction items with regard to the size of the basket (index+1)
  #Loop throught the transactions line by line e.g. first line sa = {'D', 'C', 'A'} and create combinations of them
  for sa in sets:
    getAllCombinationsOfSet = set(frozenset(i) for i in it.combinations(sa, 1+index)) #create various combination of the transction items with regard to the size of the basket (index+1)

    ###################################### NEW PART: START
    getAllCombinationsOfSet_copy = getAllCombinationsOfSet.copy() #make a copy so that we can discard supersets of which subsets have already been discarded
    #loop through all combinations 
    for i in getAllCombinationsOfSet:
      for trans in i:
        if frozenset([trans]) not in resultdictionary.keys(): #resultdictionary are the k-1 subsets, the next batch is saved in the end of this script-block
          getAllCombinationsOfSet_copy.discard(i)  #delete supersets of which subsets have already been discarded

    if runExampleFromClass:
      print("============== BEFORE DISCARDING SUPERSETS ======================")
      print(getAllCombinationsOfSet)
      print("============== AFTER DISCARDING SUPERSETS ======================")
      print(getAllCombinationsOfSet_copy)
    ###################################### NEW PART: END

    # example {'D', 'C', 'A'} and basket size 2 (1+[index=1]) = {{'D', 'C'}, {'C', 'A'}, {'D', 'A'}
    intersection = (comb_keys & getAllCombinationsOfSet_copy) # now, check if any of the combinations of the transactions intersect with all of the variuos combinationsets
    for commonKey in intersection:
      combinationsets[commonKey] += 1 #


  itemSetsOverThreshold = {} #empty it for every loop
  #save the pairs that are over the threshold
  for itemset, sup in combinationsets.items():
      if sup >= s_threshold:
        itemSetsOverThreshold[itemset] = sup # save the ones over the threshold

  result+=[itemSetsOverThreshold] 

  for itemset, sup in itemSetsOverThreshold.items():
    resultdictionary[itemset] = sup 

In [ ]:
result[-2:] #don't show singletons, only doubletons and triplet

[{frozenset({'368', '682'}): 1193,
  frozenset({'368', '829'}): 1194,
  frozenset({'39', '825'}): 1187,
  frozenset({'704', '825'}): 1102,
  frozenset({'39', '704'}): 1107,
  frozenset({'227', '390'}): 1049,
  frozenset({'390', '722'}): 1042,
  frozenset({'217', '346'}): 1336,
  frozenset({'789', '829'}): 1194},
 {frozenset({'39', '704', '825'}): 1035}]

Generating association rules with confidence at least c from the itemsets found in the first step.

In [ ]:
conf_threshold=0.5
rules = []

for idx, itemsets in enumerate(result[1:]): #run through doubletons, triplets
  # e.g doubletons-set {{'C', 'A'}: 2, {'E', 'C'}: 2, {'B', 'C'}: 2, {'E', 'B'}: 3}
  if runExampleFromClass:
    print("itemset: "+ str(itemsets)) 
  for itemset, support in itemsets.items(): #set the number of transactions to the support needed as per slides 
    # e.g itemset = {'C', 'A'} and support = 2
    if runExampleFromClass:    
      print("itemset: "+ str(itemset)) 
    for i in range(len(itemset) - 1): #loop over each item in the itemset
      if runExampleFromClass:
        print("range of itemset: "+ str(i+1)) 
      for combo in list(it.combinations(itemset, i + 1)): #Itertools to create every combinations of itemset of lenght i+1
        if runExampleFromClass:  
          print("combo: " + str(combo))
        combo = frozenset(combo) #make it immutable 
        if runExampleFromClass:
          print("if: " + str(result[len(combo) - 1][combo]))
        
        if result[len(combo) - 1][combo] is not None: #if there is a support value at e.g. result[0][frozenset('C')] = 3 then calculate confidence
          confidence = support / result[len(combo) - 1][combo] #calculate confidence
          if runExampleFromClass:  
            print("confidence: " + str(confidence))
          
          if confidence >= conf_threshold: #check if confidence is over threshold
            rules.append((combo, itemset - combo, confidence, support))
            if runExampleFromClass:  
              print("rules- combo:" + str(combo)+" itemset wihtout combo: " +str(itemset - combo) +" confidence: " +str(confidence)+" support: " +str(support)  )  

In [ ]:
#The following itemsets appear more than 50% of the time; If <rules> appears in a basket then they are likely to contain.. 
for i in rules: 
  print(i)

(frozenset({'704'}), frozenset({'825'}), 0.6142697881828316, 1102)
(frozenset({'704'}), frozenset({'39'}), 0.617056856187291, 1107)
(frozenset({'227'}), frozenset({'390'}), 0.577007700770077, 1049)
(frozenset({'704'}), frozenset({'825', '39'}), 0.5769230769230769, 1035)
(frozenset({'825', '704'}), frozenset({'39'}), 0.9392014519056261, 1035)
(frozenset({'825', '39'}), frozenset({'704'}), 0.8719460825610783, 1035)
(frozenset({'704', '39'}), frozenset({'825'}), 0.9349593495934959, 1035)
